# Exploratory Data Analysis for Olist Ecommerce Dataset
This notebook will explore the Olist Ecommerce Dataset to identify key characteristics useful for predicting customer churn.

First, import libraries and configure dataset directory.

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

DATA_DIR = 'archive'

Load each dataset as a pandas DataFrame.

In [98]:
print("Loading datasets...")

try:
    customers_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_customers_dataset.csv'))
    orders_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))
    order_items_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
    order_payments_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_payments_dataset.csv'))
    # product_info needs joining orders->items->products later
    products_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))
    # reviews might be useful for features later
    reviews_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_reviews_dataset.csv'))

    print("Datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")
    print(f"Please ensure the CSV files are in the '{DATA_DIR}' directory.")

Loading datasets...
Datasets loaded successfully.


Get a first look at each dataframe.

In [99]:
dataframes = {
    "Customers": customers_df,
    "Orders": orders_df,
    "Order Items": order_items_df,
    "Order Payments": order_payments_df,
    "Products": products_df,
    "Reviews": reviews_df
}

for name, df in dataframes.items():
    print(f"\n--- Inspecting: {name} DataFrame ---")
    print(f"Shape: {df.shape}") # (rows, columns)

    print("\nFirst 5 rows:")
    # Display more columns if needed: pd.set_option('display.max_columns', None)
    print(df.head())

    print("\nInfo (Data Types & Non-Null Counts):")
    # This is crucial for spotting missing values and wrong data types
    df.info()

    # Get basic statistics for numerical columns, only if they exist
    #print("\nDescriptive Statistics (Numerical Columns):")
    #numerical_cols = df.select_dtypes(include=np.number).columns
    #if not numerical_cols.empty:
    #    print(df.describe(include=[np.number])) # Use include=[np.number] to only show numerical stats initially
    #else:
    #    print("No numerical columns found in this DataFrame.")


    # Optional: Look at categorical descriptions if needed later
    # print("\nDescriptive Statistics (Categorical Columns):")
    # print(df.describe(include=['object']))

    print("-" * 50)


--- Inspecting: Customers DataFrame ---
Shape: (99441, 5)

First 5 rows:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  

Info (Data Types & Non-Null Counts):
<class 'pandas.core.frame.DataFrame'>
Ran

### Most Important Features Identified
- #### Recency: Days since customers last purchase
    - **Reasoning:**  Customers who haven't bought in a long time are likely to have churned
    - **Extraction:** Max of order_purchase_timestamp
- #### Frequency: Total number of orders a customer has placed
    - **Reasoning:** Customers who buy often are more engaged, less likely to churn
    - **Extraction:** Count unique order_ids associated with customer
- #### Tenure: Days since customers first purchase
    - **Reasoning:** Helps identify low-risk, loyal customers that are less likely to churn
    - **Extraction:** Days since first order date associated with customer
- #### Monetary: Total amount of money customer has spent
    - **Reasoning:** High-spenders are valuable, and their spending habit is a strong engagement signal
    - **Extraction:** Sum of all order payment values associated with customer
- #### Average Payment Installments
    - **Reasoning:** Customers who pay in many installments likely place high value orders and experience different churn patterns
    - **Extraction:** Average of all payment installment amounts for orders associated with customer
- #### Preferred Payment Method
    - **Reasoning:** Customers may have different curn patterns based on preferred payment type
    - **Extraction:** Mode of all payment_type associated with customer
- #### Average Payment Complexity: Unique payment methods per single order
    - **Reasoning:** Customers who use multiple payment methods per order likely wait for vouchers and have a different churn profile
    - **Extraction:** Average of the max payment_sequential for each order associated with a customer
- #### Average Review Score
    - **Reasoning:** Customers who leave high reviews are more satisfied, less likely to churn
    - **Extraction:** Average of all reviews associated with customer
- #### Review Engagement Rate: How often does customer leave review messages
    - **Reasoning:** A customer who leaves review messages, good or bad, is more engaged and less likely to churn
    - **Extraction:** The ratio of orders with messages vs orders without messages for orders associated wtih customer
- #### Average Shipping Cost
    - **Reasoning:** Customers who pay more for shipping are more likely to churn
    - **Extraction:** Take the average sum of freight_value for every order item for orders associated with customer
- #### Number of Unique Product Categories Purchased
    - **Reasoning:** Customers who have bought products from more categories are more invested in the platform, less likely to churn
    - **Extraction:** Number of unique product categories associated with customer orders
- #### Number of Unique Sellers Purchased From
    - **Reasoning:** Customers who purchase form many sellers are more engaged, less likely to churn
    - **Extraction:** Total of unique seller_id for orders items associated with customer
- #### Average Payment Approval Time: Gap between order placement and approval
    - **Reasoning:** Customers who frequently experience delays between placing orders and order approval are more likely to churn
    - **Extraction:** Average difference between order approval and order placement
- #### Average Delivery vs Estimate
    - **Reasoning:** Customers who consistently get late packages are likely to churn
    - **Extraction:** Average time between order_delivered_customer_date and order_estimated_delivery_date
- #### Average Carrier Transit Time
    - **Reasoning:** Customers who experience long shipping times are more likely to churn
    - **Extraction:** Average difference between carier delivery and customer delivery of all orders associated with a customer
- #### Average Time Between Seller Shipping Deadline and Carrier Delivery
    - **Reasoning:** Sellers not meeting their fulfillment deadlines will make customers more liekly to churn
    - **Extraction:** Average difference between seller deadline and carrier delivery for orders associated with customer

### Data Cleaning
- The previous output shows that dates are stored as strings, they will need to be converted to datetime objects. 
- There are some missing orders that will need filtered out. 
- Products with missing categories will need to have their categories set to 'unknown'. 
- Can remove customer_zip_code_prefix, customer_city, customer_state, order_item_id, review_id, review_comment_title, all product columns except product_id and product_category.


In [100]:
# --- 1. Clean and Minimize Orders DataFrame ---
try:
    print(f"Original shape: {orders_df.shape}")

    # Investigate 'order_status'
    print("\n'order_status' counts:")
    print(orders_df['order_status'].value_counts(dropna=False))

    # Filter for 'delivered' orders
    delivered_orders_df = orders_df[orders_df['order_status'] == 'delivered'].copy()
    print(f"\nFiltered for 'delivered' status. New shape: {delivered_orders_df.shape}")

    # Convert Timestamps
    print("\nConverting timestamp columns...")
    timestamp_cols = [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date'
    ]
    for col in timestamp_cols:
        delivered_orders_df[col] = pd.to_datetime(delivered_orders_df[col])
    
    # --- Create a Minimal DataFrame for Merging ---
    cleaned_orders_minimal = delivered_orders_df[['order_id','customer_id','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']]

    # Verify changes
    print("\nVerifying 'cleaned_orders_minimal' Dtypes:")
    cleaned_orders_minimal.info()

    # Save the cleaned file
    output_file_orders = f'{DATA_DIR}/cleaned_orders_minimal.csv'
    cleaned_orders_minimal.to_csv(output_file_orders, index=False)
    print(f"\nSuccessfully saved cleaned minimal orders data to: {output_file_orders}")
    dataframes["Orders"] = cleaned_orders_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")


# --- 2. Clean and Minimize Reviews DataFrame ---
print("\n--- Processing: Reviews DataFrame ---")
try:
    print(f"Original shape: {reviews_df.shape}")
    print("Converting timestamp columns...")

    review_ts_cols = ['review_creation_date', 'review_answer_timestamp']
    for col in review_ts_cols:
        reviews_df[col] = pd.to_datetime(reviews_df[col])

    cleaned_reviews_minimal = reviews_df[['order_id','review_score', 'review_comment_message']]
    
    # Verify changes
    print("\nVerifying 'cleaned_reviewes_minimal' Dtypes:")
    cleaned_reviews_minimal.info()

    # Save the cleaned file
    output_file_reviews = f'{DATA_DIR}/cleaned_reviews_minimal.csv'
    cleaned_reviews_minimal.to_csv(output_file_reviews, index=False)
    print(f"\nSuccessfully saved cleaned minimal reviews data to: {output_file_reviews}")
    dataframes["Reviews"] = cleaned_reviews_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")


# --- 3. Clean and Minimize Order Items DataFrame ---
print("\n--- Processing: Order Items DataFrame ---")
try:
    print(f"Original shape: {order_items_df.shape}")
    print("Converting 'shipping_limit_date' to datetime object...")
    
    order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

    cleaned_order_items_minimal = order_items_df[['order_id','product_id','seller_id','shipping_limit_date','price','freight_value']]
    
    # Verify changes
    print("\nVerifying 'cleaned_order_items_minimal' Dtypes:")
    cleaned_order_items_minimal.info()

    # Save the cleaned file
    output_file_items = f'{DATA_DIR}/cleaned_order_items_minimal.csv'
    cleaned_order_items_minimal.to_csv(output_file_items, index=False)
    print(f"\nSuccessfully saved cleaned minimal order items data to: {output_file_items}")
    dataframes["Order Items"] = cleaned_order_items_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 4. Clean and Minimize Products DataFrame ---
try:
    print(f"Original shape: {products_df.shape}")

    # --- 1. Inspect Missing Values (Before) ---
    missing_count = products_df['product_category_name'].isnull().sum()
    print(f"\nMissing 'product_category_name' values (Before): {missing_count}")

    # --- 2. Clean the Column ---
    # Fill NaN values with the string 'unknown'
    products_df['product_category_name'] = products_df['product_category_name'].fillna('unknown')
    print("Filled NaN values with 'unknown'.")

    # --- 3. Verify the Cleaning (After) ---
    missing_count_after = products_df['product_category_name'].isnull().sum()
    print(f"Missing 'product_category_name' values (After): {missing_count_after}")

    # --- 4. Create a Minimal DataFrame for Merging ---
    cleaned_products_minimal = products_df[['product_id', 'product_category_name']]
    
    print("\nHead of the cleaned, minimal products DataFrame:")
    print(cleaned_products_minimal.head())

    # --- 5. Save the Cleaned File ---
    output_file = f'{DATA_DIR}/cleaned_products_minimal.csv'
    cleaned_products_minimal.to_csv(output_file, index=False)
    print(f"\nSuccessfully saved cleaned minimal product data to: {output_file}")
    dataframes["Products"] = cleaned_products_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 5. Minimize Customer DataFrame ---
try:
    print(f"Original shape: {customers_df.shape}")

    # --- 1. Create a Minimal DataFrame for Merging ---
    customers_minimal = customers_df[['customer_id', 'customer_unique_id']]
    
    print("\nHead of the minimal customer DataFrame:")
    print(customers_minimal.head())

    # --- 2. Save the Minimal File ---
    output_file = f'{DATA_DIR}/customers_minimal.csv'
    customers_minimal.to_csv(output_file, index=False)
    print(f"\nSuccessfully saved minimal customer data to: {output_file}")
    dataframes["Customers"] = customers_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

print("\n--- All cleaning tasks complete. ---")

Original shape: (99441, 8)

'order_status' counts:
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

Filtered for 'delivered' status. New shape: (96478, 8)

Converting timestamp columns...

Verifying 'cleaned_orders_minimal' Dtypes:
<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96478 non-null  object        
 1   customer_id                    96478 non-null  object        
 2   order_purchase_timestamp       96478 non-null  datetime64[ns]
 3   order_approved_at              96464 non-null  datetime64[ns]
 4   order_delivered_carrier_date   96476 non-null  datetime64[ns]
 5   order_delivered_customer_date  9

Now that the dataframes are cleaned and minimized, it is time to calculate features and assemble the final customer centric dataframe. We'll start by calculating the recency, frequency, and tenure features.

In [101]:
# Join customers and orders to link customer_unique_id to order_id
dataframes["Customer Orders"] = pd.merge(
    dataframes["Customers"],  # Has customer_id, customer_unique_id
    dataframes["Orders"],     # Has customer_id, order_id, and all dates
    on='customer_id'
)

# We need a "snapshot date" to calculate recency (1 day after the last purchase)
snapshot_date = dataframes["Customer Orders"]['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

# Group by customer and aggregate
rft_features = dataframes["Customer Orders"].groupby('customer_unique_id').agg(
    last_purchase=('order_purchase_timestamp', 'max'),
    Frequency=('order_id', 'nunique'),
    first_purchase=('order_purchase_timestamp', 'min')
).reset_index()

# Calculate Recency and Tenure in days
rft_features['Recency'] = (snapshot_date - rft_features['last_purchase']).dt.days
rft_features['Tenure'] = (snapshot_date - rft_features['first_purchase']).dt.days

# This is our starting point for the final DataFrame
dataframes["Customer Centric"] = rft_features[['customer_unique_id', 'Recency', 'Frequency', 'Tenure']]

Now moving on to the monetary and payment features...

In [102]:
# Link payments to the customer/order master
dataframes["Merged Payments"] = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    dataframes["Order Payments"],
    on='order_id'
)

# 1. Aggregate basic payment features
payment_features = dataframes["Merged Payments"].groupby('customer_unique_id').agg(
    Monetary=('payment_value', 'sum'),
    avg_payment_installments=('payment_installments', 'mean'),
    preferred_payment_method=('payment_type', lambda x: x.mode()[0]) # Get the most frequent
).reset_index()

# 2. Calculate Payment Complexity (multi-step)
# Find the max sequential number for *each order*
order_complexity = dataframes["Merged Payments"].groupby(['customer_unique_id', 'order_id']) \
                                  .agg(max_sequential=('payment_sequential', 'max')) \
                                  .reset_index()
# Now, find the average complexity *per customer*
customer_complexity = order_complexity.groupby('customer_unique_id') \
                                      .agg(avg_payment_complexity=('max_sequential', 'mean')) \
                                      .reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], payment_features, on='customer_unique_id', how='left')
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], customer_complexity, on='customer_unique_id', how='left')

Next review features...

In [103]:
reviews_merged = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    reviews_df,
    on='order_id'
)

# Aggregate review features
review_features = reviews_merged.groupby('customer_unique_id').agg(
    avg_review_score=('review_score', 'mean'),
    # .mean() on a boolean (notnull()) gives the ratio/rate
    review_engagement_rate=('review_comment_message', lambda x: x.notnull().mean()) 
).reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], review_features, on='customer_unique_id', how='left')

Item, product, and seller features...

In [104]:
# Link items to customers/orders
items_merged = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    dataframes["Order Items"],
    on='order_id'
)
# Now link to products to get the category
products_merged = pd.merge(
    items_merged,
    products_df,
    on='product_id'
)

# Aggregate features from this combined table
item_features = products_merged.groupby('customer_unique_id').agg(
    total_freight_value=('freight_value', 'sum'), # We'll calculate avg cost later
    unique_product_categories=('product_category_name', 'nunique'),
    unique_sellers=('seller_id', 'nunique')
).reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], item_features, on='customer_unique_id', how='left')

# Create the Average Shipping Cost (using total_freight and Frequency)
dataframes["Customer Centric"]['avg_shipping_cost'] = (
    dataframes["Customer Centric"].pop('total_freight_value') / dataframes["Customer Centric"]['Frequency']
)

Gap time features...

In [105]:
# We need to link items to get the shipping_limit_date for each order
gaps_merged = pd.merge(
    dataframes["Customer Orders"],
    dataframes["Order Items"][['order_id', 'shipping_limit_date']],
    on='order_id'
)

# De-duplicate: An order with 3 items will have 3 rows. We only need one row per order.
# We'll take the LAST shipping_limit_date as the deadline for the whole order.
gaps_df_orders = gaps_merged.groupby('order_id').agg({
    'customer_unique_id': 'first',
    'order_purchase_timestamp': 'first',
    'order_approved_at': 'first',
    'order_delivered_carrier_date': 'first',
    'order_delivered_customer_date': 'first',
    'order_estimated_delivery_date': 'first',
    'shipping_limit_date': 'max' # Use the last deadline
}).reset_index()

# Calculate all gaps in hours (or days)
gaps_df_orders['avg_payment_approval_time'] = (gaps_df_orders['order_approved_at'] - gaps_df_orders['order_purchase_timestamp']).dt.total_seconds() / 3600
gaps_df_orders['avg_delivery_vs_estimate'] = (gaps_df_orders['order_estimated_delivery_date'] - gaps_df_orders['order_delivered_customer_date']).dt.total_seconds() / (24 * 3600)
gaps_df_orders['avg_carrier_shipping_time'] = (gaps_df_orders['order_delivered_customer_date'] - gaps_df_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)
gaps_df_orders['avg_shipping_vs_deadline'] = (gaps_df_orders['shipping_limit_date'] - gaps_df_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)

# Now, average these order-level gaps for each customer
gap_features = gaps_df_orders.groupby('customer_unique_id')[[
    'avg_payment_approval_time', 'avg_delivery_vs_estimate', 'avg_carrier_shipping_time',
    'avg_shipping_vs_deadline'
]].mean().reset_index()

# --- Merge this final group ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], gap_features, on='customer_unique_id', how='left')

And we're done! Time to clean any NaNs.

In [106]:
# Check for NaNs
print("NaNs before cleaning:")
print(dataframes["Customer Centric"].isnull().sum())

# Fill NaNs with logical defaults
dataframes["Customer Centric"]['avg_review_score'] = dataframes["Customer Centric"]['avg_review_score'].fillna(
    dataframes["Customer Centric"]['avg_review_score'].mean() # Impute with the mean score
)
dataframes["Customer Centric"]['review_engagement_rate'] = dataframes["Customer Centric"]['review_engagement_rate'].fillna(0) # 0% engagement
dataframes["Customer Centric"]['avg_payment_complexity'] = dataframes["Customer Centric"]['avg_payment_complexity'].fillna(1) # Default to 1

# Drop customer with no monetary value (never paid)
dataframes["Customer Centric"].dropna(subset=['Monetary'], inplace=True)

# Handle the 2-13 gap time NaNs
# Impute with the mean time for each
for col in ['avg_payment_approval_time', 'avg_delivery_vs_estimate', 
            'avg_carrier_shipping_time', 'avg_shipping_vs_deadline']:
    dataframes["Customer Centric"][col] = dataframes["Customer Centric"][col].fillna(
        dataframes["Customer Centric"][col].mean()
    )

print("NaNs after cleaning:")
print(dataframes["Customer Centric"].isnull().sum())

# Save the final file
dataframes["Customer Centric"].to_csv('customer_centric_features.csv', index=False)

print("--- Final Customer-Centric DataFrame Assembled ---")
print(dataframes["Customer Centric"].head())
print(dataframes["Customer Centric"].info())

NaNs before cleaning:
customer_unique_id             0
Recency                        0
Frequency                      0
Tenure                         0
Monetary                       1
avg_payment_installments       1
preferred_payment_method       1
avg_payment_complexity         1
avg_review_score             603
review_engagement_rate       603
unique_product_categories      0
unique_sellers                 0
avg_shipping_cost              0
avg_payment_approval_time     13
avg_delivery_vs_estimate       8
avg_carrier_shipping_time      9
avg_shipping_vs_deadline       2
dtype: int64
NaNs after cleaning:
customer_unique_id           0
Recency                      0
Frequency                    0
Tenure                       0
Monetary                     0
avg_payment_installments     0
preferred_payment_method     0
avg_payment_complexity       0
avg_review_score             0
review_engagement_rate       0
unique_product_categories    0
unique_sellers               0
avg_shippin